In [36]:
%reload_ext autoreload
%autoreload 2

In [37]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from gensim import corpora, models

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/habte/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/habte/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [39]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [49]:
sl = SlackDataLoader("/home/habte/anonymized_new/anonymized")
df = sl.slack_parser("/home/habte/anonymized_new/anonymized")
#print(df.shape)


Error: channels.json file not found.
Error: users.json file not found.


ValueError: No objects to concatenate

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
def get_top_20_user(data, channel='Random'):
    """get user with the highest number of message sent to any channel"""

    data['sender_name'].value_counts()[:20].plot.bar(figsize=(15, 7.5))
    plt.title(f'Top 20 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

    data['sender_name'].value_counts()[-10:].plot.bar(figsize=(15, 7.5))
    plt.title(f'Bottom 10 Message Senders in #{channel} channels', size=15, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=14);
    plt.xticks(size=12); plt.yticks(size=12);
    plt.show()

def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()
def top_10_senders_eda(df):

    # Find the top 10 senders by maximum reply_count
    top_10_senders = df.groupby('sender_name')['reply_count'].max().nlargest(10).reset_index()

    # Plot a bar chart for the top 10 senders by maximum reply_count
    plt.figure(figsize=(12, 6))
    sns.barplot(x='sender_name', y='reply_count', data=top_10_senders)
    plt.title('Top 10 Senders by Maximum Reply Count')
    plt.xlabel('Sender Name')
    plt.ylabel('Maximum Reply Count')
    plt.xticks(rotation=45)
    plt.show()

def bottom_10_senders_eda(df):

    # Find the top 10 senders by maximum reply_count
    bottom_10_senders_eda = df.groupby('sender_name')['reply_count'].min().nsmallest(10).reset_index()

    # Plot a bar chart for the top 10 senders by maximum reply_count
    plt.figure(figsize=(12, 6))
    sns.barplot(x='sender_name', y='reply_count', data=bottom_10_senders_eda)
    plt.title('Bottom 10 Senders by Maximum Reply Count')
    plt.xlabel('Sender Name')
    plt.ylabel('Maximum Reply Count')
    plt.xticks(rotation=45)
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:

df_community = sl.slack_parser('/home/habte/week0/data/anonymized_new /anonymized')

In [ ]:
df_community.head()

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel


In [ ]:
# Inside the slack_parser method after creating the DataFrame
print("DataFrame Info:")

print(df_community.info())


print("Sample Rows:")
print(df_community.head())

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   msg_type           0 non-null      object
 1   msg_content        0 non-null      object
 2   sender_name        0 non-null      object
 3   msg_sent_time      0 non-null      object
 4   msg_dist_type      0 non-null      object
 5   time_thread_start  0 non-null      object
 6   reply_count        0 non-null      int64 
 7   reply_users_count  0 non-null      object
 8   reply_users        0 non-null      object
 9   tm_thread_end      0 non-null      object
 10  channel            0 non-null      object
dtypes: int64(1), object(10)
memory usage: 132.0+ bytes
None
Sample Rows:
Empty DataFrame
Columns: [msg_type, msg_content, sender_name, msg_sent_time, msg_dist_type, time_thread_start, reply_count, reply_users_count, reply_users, tm_thread_end, channel]
Index: []


In [ ]:
### get user with the highest number of message sent to any channel
#get_top_20_user(df_community, channel='all-community-building')

In [ ]:
# Average number of reply count per sender
#draw_avg_reply_count(df_community, channel='all-community-building')

In [ ]:
# Visualize reply counts per user per channel
msg_content = sl.get_channel_messages(channel_name='all-community-building')

In [ ]:
# Average number of reply count per sender
#draw_avg_reply_count(df_community, channel='all-community-building')

In [ ]:
max_reply_count= df_community['reply_count'].max()
max_reply_count

In [ ]:
# Find the index of the maximum reply_count
max_reply_index = df_community['reply_count'].idxmax()

# Get the user with the maximum reply_count
user_with_max_reply = df_community.loc[max_reply_index, 'sender_name']

print(f"The user with the maximum reply_count is: {user_with_max_reply}")

In [ ]:
# Find the top 10 senders by maximum reply_count
top_10_senders = df_community.groupby('sender_name')['reply_count'].max().nlargest(10).reset_index()
# top_10_users = df.nlargest(10, 'reply_count')
print("Top 10 users by reply_count:")
print(top_10_senders[['sender_name', 'reply_count']])

In [ ]:
# Bottom 10 users by reply_count
bottom_10_users = df_community.groupby('sender_name')['reply_count'].min().nsmallest(10).reset_index()
print("\nBottom 10 users by reply_count:")
print(bottom_10_users[['sender_name', 'reply_count']])

In [ ]:
# histogram plot for bottom_10_users
bottom_10_senders_eda(df_community)

In [ ]:
df_community.shape

In [ ]:
# let's visualize it for all-community-building channel
# Aggregate reply counts per user
pivot_df = df_community.groupby('sender_name')['reply_count'].sum().reset_index()

# Plot the bar chart
plt.figure(figsize=(12, 6))
plt.bar(pivot_df['sender_name'], pivot_df['reply_count'], color='skyblue')
plt.xlabel('User')
plt.ylabel('Reply Count')
plt.title('Reply Counts per User for channel all-community-building')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()
plt.show()

In [ ]:
# Convert 'msg_sent_time' to datetime with Unix timestamp format
df_community['msg_sent_time'] = pd.to_datetime(df_community['msg_sent_time'], unit='s')

# Extract the hour from the 'msg_sent_time' column
df_community['hour'] = df_community['msg_sent_time'].dt.hour

# Plot a histogram to visualize the distribution of messages across hours
plt.figure(figsize=(12, 6))
df_community['hour'].plot(kind='hist', bins=24, rwidth=0.8, color='#86bf91', alpha=0.7)
plt.title('Distribution of Messages Across Hours of the Day')
plt.xlabel('Hour of the Day')
plt.ylabel('Number of Messages')
plt.xticks(range(24))
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Find the hour range with the highest message count
most_active_hour_range = df_community['hour'].value_counts().idxmax()
#print(f"The time range of the day with the most messages is approximately {most_active_hour_range}:00 to {(mos)}")

In [ ]:
# Convert 'msg_sent_time' and 'tm_thread_end' to datetime
df_community['msg_sent_time'] = pd.to_datetime(df_community['msg_sent_time'], unit='s')
df_community['time_thread_start'] = pd.to_datetime(df_community['time_thread_start'], unit='s')

# Calculate the time taken to reply to each message
df_community['reply_time'] = (df_community['time_thread_start'] - df_community['msg_sent_time']).dt.total_seconds()

average_reply_time_per_type = df_community.groupby('msg_type')['reply_time'].mean()

# Plot a bar chart
plt.figure(figsize=(12, 6))
average_reply_time_per_type.plot(kind='bar', color='#86bf91', alpha=0.7)
plt.title('Average Reply Time per Message Type')
plt.xlabel('Message Type')
plt.ylabel('Average Reply Time (seconds)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
 #Relationship between # of messages and # of reactions

    
df_reaction = sl.parse_slack_reaction(channel='all-technical-support',path_channel='../Anonymized_B6SlackExport_25Nov23/anonymized/all-technical-support/')

In [ ]:
df_reaction.columns

In [ ]:
# Function to classify messages
def classify_message(msg):
    # Tokenize the message
    words = word_tokenize(msg.lower())  # Convert to lowercase for case-insensitivity
    
    # Remove stopwords
    filtered_words = [word for word in words if word.isalnum() and word not in stopwords.words('english')]
    
    # Example classification logic (customize based on your needs)
    if '?' in filtered_words:
        return 'Question'
    elif 'thanks' in filtered_words or 'thank you' in filtered_words:
        return 'Thanks'
    elif 'nice' in filtered_words or 'good job' in filtered_words:
        return 'Appreciation'
    else:
        return 'Other'

# Apply the classification function to each message in the DataFrame
df_community['message_category'] = df_community['msg_content'].apply(classify_message)

In [ ]:
category_counts = df_community['message_category'].value_counts()

# Plot a bar chart
category_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Message Category')
plt.ylabel('Count')
plt.title('Message Category Distribution')
plt.show()

In [ ]:
user_reactions = df_reaction.groupby('user_id')['reaction_count'].sum()

# Sort the results to get the users with the most reactions
user_reactions = user_reactions.sort_values(ascending=False)

# Display the user_id and reaction_count for the top users
top_users_reactions = user_reactions.reset_index()
print(top_users_reactions[['user_id', 'reaction_count']])

In [ ]:
user_id_name_dict = sl.get_user_map()
# Access the dictionary within the tuple
user_id_name_dict = user_id_name_dict[0]

# Create a DataFrame from the dictionary
df_users = pd.DataFrame(list(user_id_name_dict.items()), columns=['user_id', 'name'])
# Merge the two DataFrames on the 'user_id' column
df_merged = pd.merge(top_users_reactions, df_users, on='user_id')

# Print the merged DataFrame
print(df_merged)

In [ ]:
plt.figure(figsize=(10, 6))
plt.bar(df_merged['name'], df_merged['reaction_count'], color='skyblue')
plt.xlabel('User Name')
plt.ylabel('Reaction Count')
plt.title('Reaction Counts by User')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?